##  A 3-Hidden Layers Architecture (Broad Search - Parametric Study)
What is happening here?
    [1] A deep DNN model
    [2] Contains two lines of code to find the rsquare
    [3] Includes lines of code on cross validation
    [4] Include the plot of the history curves
    
The lower value of MAE, MSE, and RMSE implies higher accuracy of a regression model. However, a higher value of R square is considered desirable.

In [1]:
import pandas as pd 
import numpy as np                                             
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam
from sklearn.metrics import r2_score
import math

# Load the data
bucklingdata = pd.read_csv("CBL_SimulationResults.csv", encoding='cp1252')
input_data = bucklingdata.drop(['Critical Buckling Load (N)', 'Critical Buckling Load (kN)'], axis=1)
output_data = bucklingdata['Critical Buckling Load (kN)']

# Split the data
np.random.seed(0)
X_train, X_test, y_train, y_test = train_test_split(input_data, output_data,
                                                    test_size=0.3,
                                                    shuffle=False,
                                                    stratify=None,
                                                    random_state=42)

# Validation data
val_no = round(0.2 * y_train.size)
x_val = X_train[:val_no]
y_val = y_train[:val_no]

def create_model3H(hp_layer_1, hp_layer_2, hp_layer_3):
    model = Sequential()
    model.add(Input(shape=(11,)))
    hp_activation = 'selu'
    hp_learning_rate = 0.001
    model.add(Dense(hp_layer_1, activation=hp_activation))
    model.add(Dense(hp_layer_2, activation=hp_activation))
    model.add(Dense(hp_layer_3, activation=hp_activation))
    model.add(Dense(1, activation="linear"))
    model.compile(loss='mse',
                  optimizer=Adam(learning_rate = hp_learning_rate),
                  metrics=["mae"])
    return model

# Specify the range for neurons in the three hidden layers
min_neurons = 20
max_neurons = 100
step = 10

neuron_configs = []

# Generate configurations with increasing neurons in all three layers
for neurons1 in range(min_neurons, max_neurons + 1, step):
    for neurons2 in range(min_neurons, max_neurons + 1, step):
        for neurons3 in range(min_neurons, max_neurons + 1, step):
            neuron_configs.append((neurons1, neurons2, neurons3))
            
results = []

## Train your model for a number of epochs, with the .fit()
## Get the performance metrics for the network and the learning curves
# Iterate over each configuration of neurons
for i, (p1, p2, p3) in enumerate(neuron_configs):
    print(f"Training model with configuration {i+1}: Hidden Layer 1={p1}, Hidden Layer 2={p2}, Hidden Layer 3={p3}")
    
    # Create the model for the current configuration
    model = create_model3H(p1, p2, p3)
    
    # Fit the model
    history = model.fit(X_train, y_train, epochs=150,
                        validation_data=(x_val, y_val), batch_size=32,
                        verbose=0)
    
    # Evaluate the model on training, validation, and test sets
    trainmse, trainmae = model.evaluate(X_train, y_train, verbose=0)
    valmse, valmae = model.evaluate(x_val, y_val, verbose=0)
    testmse, testmae = model.evaluate(X_test, y_test, verbose=0)
    
    # Predictions for R-squared calculation
    train_pred = model.predict(X_train)
    val_pred = model.predict(x_val)
    test_pred = model.predict(X_test)
    
    # Calculate R-squared scores
    train_r2 = r2_score(y_train, train_pred)
    val_r2 = r2_score(y_val, val_pred)
    test_r2 = r2_score(y_test, test_pred)
    
    # Store the evaluation metrics in a DataFrame
    result_df = pd.DataFrame({
        'Configuration': i + 1,
        'Hidden Layer 1 Neurons': p1,
        'Hidden Layer 2 Neurons': p2,
        'Hidden Layer 3 Neurons': p3,
        'Train R2': train_r2,
        'Val R2': val_r2,
        'Test R2': test_r2,
        'Train RMSE': math.sqrt(trainmse),
        'Val RMSE': math.sqrt(valmse),
        'Test RMSE': math.sqrt(testmse),
        'Train MAE': trainmae,
        'Val MAE': valmae,
        'Test MAE': testmae
    }, index=[0])
    
    results.append(result_df)

# Concatenate all results into a single DataFrame
final_results = pd.concat(results, ignore_index=True)

# Save the results to an Excel file
final_results.to_excel('3HiddenLayers_VariousNeurons_Results_1stRun.xlsx', index=False)

print(f"Results saved to 3HiddenLayers_VariousNeurons_Results.xlsx")


Training model with configuration 1: Hidden Layer 1=20, Hidden Layer 2=20, Hidden Layer 3=20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training model with configuration 2: Hidden Layer 1=20, Hidden Layer 2=20, Hidden Layer 3=30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 798us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step
Training model with configuration 3: Hidden Layer 1=20, Hidden Layer 2=20, Hidden Layer 3=40
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 834us/step
Training model with configuration 4: Hidden Layer 1=20, Hidden Layer 2=20, Hidden Layer 3=50
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training model with configuration 5: Hidden Layer 1=20, Hidden Layer 2=20, Hidden Layer 3=60
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
6/6 ━━━━━━━━━━━━━━━━━

### loss vs the number of epochs & MAE vs the number of epochs

In [ ]:
def plot_loss(loss,val_loss):
  plt.figure()
  plt.style.use('bmh')
  plt.grid(False)
  plt.plot(loss, linewidth=2)
  plt.plot(val_loss, linewidth=2)
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper right')
  plt.show()

def plot_mae(mae,val_mae):
  plt.figure()
  plt.style.use('ggplot')
  plt.grid(False)
  plt.plot(mae, linewidth=2)
  plt.plot(val_mae, linewidth=2)
  plt.ylabel('MAE')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper right')
  plt.show()

train_loss = (np.array(history.history["loss"]))/100
val_loss = np.array((history.history["val_loss"]))/100

plot_loss(train_loss, val_loss)
plot_mae(history.history["mae"], history.history["val_mae"])

### Correlation between the actual output and the predicted output with the training dataset

In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
a = plt.axes(aspect='equal')
plt.scatter(y_train, train_pred, s = 20, alpha=0.7, c = 'brown')
plt.xlabel('Actual Critical Buckling Loads (kN)')
plt.ylabel('Predicted Critical Buckling Loads (kN)')
plt.annotate("R-squared = {:.3f}".format(r2_score(y_train, train_pred)), (6, 17.5))
plt.xlim([5, 20])
plt.ylim([5, 20])
plt.plot([5, 20], [5, 20])
plt.plot()

### Correlation between the actual output and the predicted output with the testing dataset

In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
a = plt.axes(aspect='equal')
plt.scatter(y_test, test_pred, s = 20, alpha=0.7, c = 'brown')
plt.xlabel('Actual Critical Buckling Loads (kN)')
plt.ylabel('Predicted Critical Buckling Loads (kN)')
plt.annotate("R-squared = {:.3f}".format(r2_score(y_train, train_pred)), (5, 19))
plt.xlim([4, 20])
plt.ylim([4, 20])
plt.plot([4, 20], [4, 20])
plt.plot()

## Variable importance SHAP plot

In [ ]:
### VARIABLE IMPORTANCE
# See more here: https://shap-lrjball.readthedocs.io/en/latest/example_notebooks/plots/bar.html
features = ["Number of holes, nh", "Web-post width, WP (mm)",  "Web opening diameter, Do (mm)",  "Ply 1 (ø)", "Ply 2 (ø)", "Ply 3 (ø)", "Ply 4 (ø)", "Ply 5 (ø)", "Ply 6 (ø)", "Ply 7 (ø)", "Ply 8 (ø)"]

# Create a SHAP explainer using KernelExplainer
explainer = shap.KernelExplainer(model, shap.sample(X_train, 1))

# Calculate SHAP values for the test dataset
shap_values = explainer.shap_values(X_test)

# Reshape shap_values to remove the last dimension
shap_values_reshaped = shap_values.reshape(X_test.shape)  # Assuming X_test.shape is (389, 11)

# Visualize feature importances using bar plot
shap.summary_plot(shap_values_reshaped, X_test, feature_names=features, plot_type="bar")
plt.show()